### Import Library

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
RAW_DATA_PATH = 'data_update'
DB_DATA_PATH = 'database'

### Utility Function

In [3]:
month_dict = {
    'januari'  : 1,
    'febuari'  : 2,
    'februari' : 2,
    'maret'    : 3,
    'april'    : 4,
    'mei'      : 5,
    'juni'     : 6,
    'juli'     : 7,
    'agustus'  : 8,
    'september': 9,
    'oktober'  : 10,
    'november' : 11,
    'desember' : 12
}

weight_growth_retail = {
    'DKI Jakarta'  :19.23,
    'Kota Bandung' :4.87,
    'Kota Surabaya':5.65,
    'Kota Medan'   :3.66,
    'Kota Semarang':0.59,
    'Kota Banjarmasin':1.24,
    'Kota Makassar':2.32,
    'Kota Manado': 0.78,
    'Kota Denpasar':1.78
}

In [4]:
def get_kuartal(bulan):
    if bulan > 0 and bulan <= 3:
        return 1
    elif bulan > 3 and bulan <= 6:
        return 2
    elif bulan > 6 and bulan <= 9:
        return 3
    else:
        return 4

In [5]:
def preprocess_data_bulanan(df, mode='mean'):
    df['Bulan']   = df['Bulan'].apply(lambda x: month_dict[str(x).lower().strip()])
    df['Quarter'] = df['Bulan'].apply(get_kuartal)
    if mode == 'mean':
        df = df.groupby(['Quarter', 'Tahun']).mean().reset_index()
    elif mode =='eop':
        last = []
        df = df.sort_values(['Bulan','Tahun'])
        max_quarter = df.tail(1)['Quarter'].values[0]
        max_month = df.tail(1)['Bulan'].values[0]
        if (max_quarter == 4 and max_month < 12) or (max_quarter == 3 and max_month < 9) or (max_quarter == 2 and max_month < 6) or (max_quarter == 1 and max_month < 3):
                last = df.tail(1)
        months = [3,6,9,12]
        df = df[df['Bulan'].isin(months)]
        print(df)
        df = pd.concat([df, last])
    df = df.drop(['Bulan'], axis=1)
    return df

In [6]:
def preprocess_data_harian(df, mode='mean'):
    df['Tahun']   = df['Tanggal'].apply(lambda x: int('20' + x.strftime("%y")))
    df['Bulan']   = df['Tanggal'].apply(lambda x: int(x.strftime("%m")))
    df['Hari']    = df['Tanggal'].apply(lambda x: int(x.strftime("%d")))
    df['Quarter'] = df['Bulan'].apply(get_kuartal)
    if mode == 'mean':
        df = df.groupby(['Quarter', 'Tahun']).mean().reset_index()
    elif mode == 'eop':
        df = df.sort_values(['Hari', 'Bulan','Tahun'])
        last = df.tail(1)
        quarter = [3,6,9,12]
        df = df[df['Bulan'].isin(quarter)]
        eod = [30,31]
        df = df[df['Hari'].isin(eod)]
        df_1 = df[df['Hari'] == 30]
        df_2 = df[df['Hari'] == 31]
        df_1 = df_1[df_1['Bulan'].isin([6,9])]
        df = pd.concat([df_1,df_2])
        df = pd.concat([df, last])
        df = df.drop(['Tanggal'], axis=1)
        df = df.drop_duplicates(subset=['Quarter', 'Tahun'], keep='first')
    df = df.drop(['Bulan', 'Hari'], axis=1)
    df = df.sort_values(by=['Tahun', 'Quarter'])
    return df

def preprocess_data_harian_pangan(df, mode='mean'):
    df['Tahun']   = df['Tanggal'].apply(lambda x: int(x.split('/')[2]))
    df['Bulan']   = df['Tanggal'].apply(lambda x: int(x.split("/")[1]))
    df['Hari']    = df['Tanggal'].apply(lambda x: int(x.split("/")[0]))
    df['Quarter'] = df['Bulan'].apply(get_kuartal)
    if mode == 'mean':
        df = df.groupby(['Quarter', 'Tahun']).mean().reset_index()
    elif mode == 'eop':
        df = df.sort_values(['Hari', 'Bulan','Tahun'])
        last = df.tail(1)
        quarter = [3,6,9,12]
        df = df[df['Bulan'].isin(quarter)]
        eod = [30,31]
        df = df[df['Hari'].isin(eod)]
        df_1 = df[df['Hari'] == 30]
        df_2 = df[df['Hari'] == 31]
        df_1 = df_1[df_1['Bulan'].isin([6,9])]
        df = pd.concat([df_1,df_2])
        df = pd.concat([df, last])
        df = df.drop(['Tanggal'], axis=1)
        df = df.drop_duplicates(subset=['Quarter', 'Tahun'], keep='first')
    df = df.drop(['Bulan', 'Hari'], axis=1)
    df = df.sort_values(by=['Tahun', 'Quarter'])
    return df

In [7]:
def update_database(db_path, new_df):
    db = pd.read_csv(db_path)
    db = pd.concat([db, new_df], sort=False)
    db = db.drop_duplicates(subset=['Tahun', 'Quarter'], keep='last')
    db.to_csv(db_path, index=False)

## Cement Consumption

In [8]:
# bulanan mean
konsumsi_semen     = pd.read_excel("{}/Volume Konsumsi Semen.xlsx".format(RAW_DATA_PATH))

In [9]:
konsumsi_semen.head()

,Tahun,Bulan,Konsumsi Semen (ton)
0,2019,Januari,5621904
1,2019,Febuari,4924977
2,2019,Maret,5173103
3,2019,April,4875625
4,2019,Mei,5151608


In [10]:
konsumsi_semen  = preprocess_data_bulanan(konsumsi_semen, 'mean')

In [11]:
konsumsi_semen.columns = ['Quarter', 'Tahun', 'Cement Consumption']

In [12]:
konsumsi_semen

,Quarter,Tahun,Cement Consumption
0,1,2019,5.239995e+06
1,2,2019,4.588661e+06
2,3,2019,6.448502e+06
3,4,2019,7.326461e+06


In [13]:
update_database("{}/1.Semen.csv".format(DB_DATA_PATH), konsumsi_semen)

## Car Sales

In [14]:
penjualan_mobil    = pd.read_excel("{}/Volume Penjuaan Mobil.xlsx".format(RAW_DATA_PATH))

In [15]:
penjualan_mobil  = preprocess_data_bulanan(penjualan_mobil, 'mean')

In [16]:
penjualan_mobil.columns = ['Quarter', 'Tahun', 'Car Sales']

In [17]:
penjualan_mobil

,Quarter,Tahun,Car Sales
0,1,2019,84598.000000
1,2,2019,75759.333333
2,3,2019,90835.666667
3,4,2019,96030.000000


In [18]:
update_database("{}/2.Mobil.csv".format(DB_DATA_PATH), penjualan_mobil)

## Motor Sales

In [19]:
penjualan_motor    = pd.read_excel("{}/Volume Penjualan Motor.xlsx".format(RAW_DATA_PATH))

In [20]:
penjualan_motor  = preprocess_data_bulanan(penjualan_motor, 'mean')

In [21]:
penjualan_motor.columns = ['Quarter', 'Tahun', 'Motorcycle Sales']

In [22]:
penjualan_motor

,Quarter,Tahun,Motorcycle Sales
0,1,2019,560484.666667
1,2,2019,515055.000000
2,3,2019,564344.000000
3,4,2019,597979.000000


In [23]:
update_database("{}/11.Motor Sales.csv".format(DB_DATA_PATH), penjualan_motor)

## Nilai Tukar Petani

In [24]:
nilai_tukar_petani = pd.read_excel("{}/Nilai Tukar Petani.xlsx".format(RAW_DATA_PATH))

In [25]:
nilai_tukar_petani  = preprocess_data_bulanan(nilai_tukar_petani, 'eop')

   Tahun  Bulan     NTP  Quarter
2   2019      3  102.73        1
5   2019      6  102.33        2
8   2019      9  103.88        3


In [26]:
nilai_tukar_petani = nilai_tukar_petani[['Quarter', 'Tahun', 'NTP']]

In [27]:
nilai_tukar_petani = nilai_tukar_petani.rename(columns={"NTP": "Nilai Tukar Petani"})

In [28]:
nilai_tukar_petani

,Quarter,Tahun,Nilai Tukar Petani
2,1,2019,102.73
5,2,2019,102.33
8,3,2019,103.88
10,4,2019,104.10


In [29]:
update_database("{}/6.NTP.csv".format(DB_DATA_PATH), nilai_tukar_petani)

## IKK

In [30]:
ikk                = pd.read_excel("{}/IKK.xlsx".format(RAW_DATA_PATH))

In [31]:
ikk  = preprocess_data_bulanan(ikk, 'eop')

   Tahun  Bulan    IKK  Quarter
2   2019      3  124.4        1
5   2019      6  126.4        2
8   2019      9  121.8        3


In [32]:
ikk = ikk[['Quarter', 'Tahun', 'IKK']]

In [33]:
ikk

,Quarter,Tahun,IKK
2,1,2019,124.4
5,2,2019,126.4
8,3,2019,121.8
10,4,2019,124.2


In [34]:
update_database("{}/4.IKK.csv".format(DB_DATA_PATH), ikk)

## IHK

In [35]:
ihk_umum_nasional  = pd.read_excel("{}/IHK Umum Nasional 2019.xlsx".format(RAW_DATA_PATH))

In [36]:
ihk_umum_nasional  = preprocess_data_bulanan(ihk_umum_nasional, 'eop')

   Tahun  Bulan  IHK Umum  Quarter
2   2019      3    135.87        1
5   2019      6    138.16        2
8   2019      9    138.37        3


In [37]:
ihk = ihk_umum_nasional[['Quarter', 'Tahun', 'IHK Umum']]

In [38]:
ihk_umum_nasional = ihk_umum_nasional.rename(columns={"IHK Umum": "IHK"})

In [39]:
ihk_umum_nasional

,Tahun,IHK,Quarter
2,2019,135.87,1
5,2019,138.16,2
8,2019,138.37,3
10,2019,138.60,4


In [40]:
update_database("{}/9.IHK.csv".format(DB_DATA_PATH), ihk_umum_nasional)

## Nilai Tukar Rupiah

In [41]:
nilai_tukar_rupiah = pd.read_excel("{}/Nilai Tukar Rupiah.xlsx".format(RAW_DATA_PATH))
nilai_tukar_rupiah = preprocess_data_harian(nilai_tukar_rupiah)

In [42]:
nilai_tukar_rupiah = nilai_tukar_rupiah.rename(columns={"Nilai Tukar": "Exchange Rate"})

In [43]:
nilai_tukar_rupiah

,Quarter,Tahun,Exchange Rate
0,1,2019,14138.355556
1,2,2019,14249.362637
2,3,2019,14117.771739
3,4,2019,14091.406250


In [44]:
update_database("{}/5.Exchange Rate.csv".format(DB_DATA_PATH), nilai_tukar_rupiah)

## IHSG

In [45]:
ihsg = pd.read_excel("{}/IHSG (1).xlsx".format(RAW_DATA_PATH))
ihsg = preprocess_data_harian(ihsg, 'eop')

In [46]:
ihsg.head()

,IHSG,Tahun,Quarter
89,6468.755,2019,1
180,6358.629,2019,2
272,6169.102,2019,3
303,6228.317,2019,4


In [47]:
update_database("{}/12.IHSG.csv".format(DB_DATA_PATH), ihsg)

## Kredit Konsumsi

In [48]:
kredit = pd.read_excel("{}/Kredit Konsumsi.xlsx".format(RAW_DATA_PATH))

In [49]:
kredit = preprocess_data_bulanan(kredit, 'eop')

   Tahun  Bulan  Kredit Konsumsi  Quarter
2   2019      3     1.487484e+06        1
5   2019      6     1.502616e+06        2
8   2019      9     1.526756e+06        3


In [50]:
kredit.head()

,Tahun,Kredit Konsumsi,Quarter
2,2019,1.487484e+06,1
5,2019,1.502616e+06,2
8,2019,1.526756e+06,3
9,2019,1.541014e+06,4


In [51]:
update_database("{}/7.Kredit Konsumsi.csv".format(DB_DATA_PATH), kredit)

## DPK

In [52]:
dpk = pd.read_excel("{}/Dana Pihak Ketiga.xlsx".format(RAW_DATA_PATH))

In [53]:
dpk = preprocess_data_bulanan(dpk, 'eop')

   Tahun  Bulan           DPK  Quarter
2   2019      3  5.672886e+06        1
5   2019      6  5.799494e+06        2
8   2019      9  5.891918e+06        3


In [54]:
dpk.head()

,Tahun,DPK,Quarter
2,2019,5.672886e+06,1
5,2019,5.799494e+06,2
8,2019,5.891918e+06,3
9,2019,5.882243e+06,4


In [55]:
update_database("{}/8.DPK.csv".format(DB_DATA_PATH), dpk)

## Minyak Dunia

In [56]:
minyak_dunia = pd.read_excel("{}/Harga Minyak Dunia.xlsx".format(RAW_DATA_PATH))

In [57]:
minyak_dunia = preprocess_data_harian(minyak_dunia, 'mean')

In [58]:
minyak_dunia.head()

,Quarter,Tahun,Minyak Dunia
0,1,2019,63.833492
1,2,2019,68.467656
2,3,2019,62.030455
3,4,2019,61.412885


In [59]:
update_database("{}/3.Minyak Dunia.csv".format(DB_DATA_PATH), minyak_dunia)

## Bawang Merah

In [60]:
bawang_merah = pd.read_excel("{}/Bawang Merah Nasional 2019.xlsx".format(RAW_DATA_PATH))

In [61]:
bawang_merah.head()

,Tanggal,Bawang Merah
0,02/01/2019,32500
1,03/01/2019,32750
2,04/01/2019,32850
3,07/01/2019,33350
4,08/01/2019,33700


In [62]:
bawang_merah['Bawang Merah'] = bawang_merah['Bawang Merah'].replace('-', 0)
bawang_merah['Bawang Merah'] = bawang_merah['Bawang Merah'].astype(float)

In [63]:
bawang_merah = preprocess_data_harian_pangan(bawang_merah, 'mean')

In [64]:
bawang_merah.head()

,Quarter,Tahun,Bawang Merah
0,1,2019,31763.934426
1,2,2019,38405.737705
2,3,2019,29178.030303
3,4,2019,27885.576923


In [65]:
update_database("{}/22.Bawang Merah.csv".format(DB_DATA_PATH), bawang_merah)

## Beras

In [66]:
beras = pd.read_excel("{}/Beras Nassional 2019.xlsx".format(RAW_DATA_PATH))

In [67]:
beras.head()

,Tanggal,Beras
0,02/01/2019,11850
1,03/01/2019,11850
2,04/01/2019,11850
3,07/01/2019,11850
4,08/01/2019,11900


In [68]:
beras['Beras'] = beras['Beras'].replace('-', 0)
beras['Beras'] = beras['Beras'].astype(float)

In [69]:
beras = preprocess_data_harian_pangan(beras, 'mean')

In [70]:
beras.head()

,Quarter,Tahun,Beras
0,1,2019,11890.163934
1,2,2019,11551.639344
2,3,2019,11710.606061
3,4,2019,11748.076923


In [71]:
update_database("{}/18.Beras.csv".format(DB_DATA_PATH), beras)

## Cabai Merah

In [72]:
cabe_merah = pd.read_excel("{}/Cabai Merah Nasional 2019.xlsx".format(RAW_DATA_PATH))

In [73]:
cabe_merah.head()

,Tanggal,Cabe Merah Biasa
0,02/01/2019,34350
1,03/01/2019,33850
2,04/01/2019,33600
3,07/01/2019,32950
4,08/01/2019,32700


In [74]:
cabe_merah['Cabe Merah Biasa'] = cabe_merah['Cabe Merah Biasa'].replace('-', 0)
cabe_merah['Cabe Merah Biasa'] = cabe_merah['Cabe Merah Biasa'].astype(float)

In [75]:
cabe_merah = preprocess_data_harian_pangan(cabe_merah, 'mean')

In [76]:
cabe_merah.head()

,Quarter,Tahun,Cabe Merah Biasa
0,1,2019,28824.590164
1,2,2019,40283.606557
2,3,2019,55893.181818
3,4,2019,40405.769231


In [77]:
update_database("{}/21.Cabe Merah Biasa.csv".format(DB_DATA_PATH), cabe_merah)

## Daging Ayam

In [78]:
daging_ayam = pd.read_excel("{}/Daging Ayam Nasional 2019.xlsx".format(RAW_DATA_PATH))

In [79]:
daging_ayam.head()

,Tanggal,Daging Ayam Broiler
0,02/01/2019,39650
1,03/01/2019,39350
2,04/01/2019,39200
3,07/01/2019,38550
4,08/01/2019,38100


In [80]:
daging_ayam['Daging Ayam Broiler'] = daging_ayam['Daging Ayam Broiler'].replace('-', 0)
daging_ayam['Daging Ayam Broiler'] = daging_ayam['Daging Ayam Broiler'].astype(float)

In [81]:
daging_ayam = preprocess_data_harian_pangan(daging_ayam, 'mean')

In [82]:
daging_ayam.head()

,Quarter,Tahun,Daging Ayam Broiler
0,1,2019,34041.803279
1,2,2019,34164.754098
2,3,2019,32958.333333
3,4,2019,33911.538462


In [83]:
update_database("{}/16.Daging Ayam Broiler.csv".format(DB_DATA_PATH), daging_ayam)

## Daging Sapi

In [84]:
daging_sapi = pd.read_excel("{}/Daging Sapi Nasional 2019.xlsx".format(RAW_DATA_PATH))

In [85]:
daging_sapi['Daging Sapi'] = daging_sapi['Daging Sapi'].replace('-', 0)
daging_sapi['Daging Sapi'] = daging_sapi['Daging Sapi'].astype(float)

In [86]:
daging_sapi = preprocess_data_harian_pangan(daging_sapi, 'mean')

In [87]:
daging_sapi.head()

,Quarter,Tahun,Daging Sapi
0,1,2019,116162.295082
1,2,2019,116166.393443
2,3,2019,118143.181818
3,4,2019,118123.076923


In [88]:
update_database("{}/15.Daging Sapi.csv".format(DB_DATA_PATH), daging_sapi)

## Gula Pasir

In [89]:
gula_pasir = pd.read_excel("{}/Gula Pasir Nasional 2019.xlsx".format(RAW_DATA_PATH))

In [90]:
gula_pasir['Gula Pasir'] = gula_pasir['Gula Pasir'].replace('-', 0)
gula_pasir['Gula Pasir'] = gula_pasir['Gula Pasir'].astype(float)

In [91]:
gula_pasir = preprocess_data_harian_pangan(gula_pasir, 'mean')

In [92]:
gula_pasir.head()

,Quarter,Tahun,Gula Pasir
0,1,2019,13146.721311
1,2,2019,13436.885246
2,3,2019,13774.242424
3,4,2019,13784.615385


In [93]:
update_database("{}/19.Gula Pasir.csv".format(DB_DATA_PATH), gula_pasir)

## Minyak Goreng

In [94]:
minyak_goreng = pd.read_excel("{}/Minyak Goreng Nasional 2019.xlsx".format(RAW_DATA_PATH))

In [95]:
minyak_goreng.head()

,Tanggal,Minyak Goreng Curah
0,02/01/2019,13200
1,03/01/2019,13200
2,04/01/2019,13200
3,07/01/2019,13200
4,08/01/2019,13200


In [96]:
minyak_goreng['Minyak Goreng Curah'] = minyak_goreng['Minyak Goreng Curah'].replace('-', 0)
minyak_goreng['Minyak Goreng Curah'] = minyak_goreng['Minyak Goreng Curah'].astype(float)

In [97]:
minyak_goreng = preprocess_data_harian_pangan(minyak_goreng, 'mean')

In [98]:
minyak_goreng.head()

,Quarter,Tahun,Minyak Goreng Curah
0,1,2019,13210.655738
1,2,2019,13031.967213
2,3,2019,13187.878788
3,4,2019,13218.269231


In [99]:
update_database("{}/14.Minyak Goreng Curah.csv".format(DB_DATA_PATH), minyak_goreng)

## Telur Ayam Ras

In [100]:
telur_ayam = pd.read_excel("{}/Telur Ayam Nasional 2019.xlsx".format(RAW_DATA_PATH))

In [101]:
telur_ayam.head()

,Tanggal,Telur Ayam Ras
0,02/01/2019,26850
1,03/01/2019,26800
2,04/01/2019,26750
3,07/01/2019,26500
4,08/01/2019,26350


In [102]:
telur_ayam['Telur Ayam Ras'] = telur_ayam['Telur Ayam Ras'].replace('-', 0)
telur_ayam['Telur Ayam Ras'] = telur_ayam['Telur Ayam Ras'].astype(float)

In [103]:
telur_ayam = preprocess_data_harian_pangan(telur_ayam, 'mean')

In [104]:
telur_ayam.head()

,Quarter,Tahun,Telur Ayam Ras
0,1,2019,24809.016393
1,2,2019,24947.540984
2,3,2019,24642.424242
3,4,2019,23868.269231


In [105]:
update_database("{}/17.Telur Ayam Ras.csv".format(DB_DATA_PATH), telur_ayam)

## Retail Growth

In [106]:
retail_growth = pd.read_excel("{}/Growth Retail Sales Index (YoY).xlsx".format(RAW_DATA_PATH))

In [107]:
retail_growth.head()

,Tahun,Wilayah,Bulan,Retail Growth
0,2019,DKI Jakarta,Januari,-17.179524
1,2019,DKI Jakarta,Februari,0.691416
2,2019,DKI Jakarta,Maret,10.500735
3,2019,DKI Jakarta,April,8.583075
4,2019,DKI Jakarta,Mei,13.691898


In [108]:
retail_growth['Weight'] = retail_growth['Wilayah'].apply(lambda x: weight_growth_retail[x])

In [109]:
retail_growth['Retail Growth'] = retail_growth['Retail Growth'] * retail_growth['Weight'] / 97.94

In [110]:
retail_growth = retail_growth.groupby(['Tahun', 'Bulan']).sum().reset_index()

In [111]:
retail_growth

,Tahun,Bulan,Retail Growth,Weight
0,2019,Agustus,2.831621,40.12
1,2019,April,4.345300,40.12
2,2019,Februari,3.504189,40.12
3,2019,Januari,0.006334,40.12
4,2019,Juli,6.022588,40.12
5,2019,Juni,3.051059,40.12
6,2019,Maret,5.484820,40.12
7,2019,Mei,4.970697,40.12
8,2019,Oktober,1.022922,40.12
9,2019,September,0.409690,40.12


In [112]:
retail_growth = retail_growth.drop(['Weight'], axis=1)

In [113]:
retail_growth = preprocess_data_bulanan(retail_growth, 'eop')

   Tahun  Bulan  Retail Growth  Quarter
6   2019      3       5.484820        1
5   2019      6       3.051059        2
9   2019      9       0.409690        3


In [114]:
retail_growth.head()

,Tahun,Retail Growth,Quarter
6,2019,5.484820,1
5,2019,3.051059,2
9,2019,0.409690,3
8,2019,1.022922,4


In [115]:
update_database("{}/13.Retail Growth.csv".format(DB_DATA_PATH), retail_growth)